In [1]:
!git clone https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


In [2]:
!cd models/research & protoc object_detection/protos/*.proto --python_out=.

In [3]:
!copy ".\models\research\object_detection\packages\tf2\setup.py" ".\models\research"

        1 file(s) copied.


In [4]:
import re
with open("./models/research/object_detection/packages/tf2/setup.py", 'r') as file:
  doc = file.read()
  
with open("./models/research/setup.py", 'w') as file:
  replacedTxt = re.sub('tf-models-official>=2.5.1', 'tf-models-official==2.8.0', doc)
  file.write(replacedTxt)

In [5]:
!pip install models/research 

Processing c:\users\zahid\source\repos\ml\project x - tensorflow\realtime object detection using tf model garden\model test\models\research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1708397 sha256=372993368acc10fd86560f00dc371835ad4d6d38cac994a458788f357929f506
  Stored in directory: C:\Users\zahid\AppData\Local\Temp\pip-ephem-wheel-cache-v4k21oqd\wheels\ba\18\db\ea5bc9250e4307c11236ceb02db800588d91a0c344ca390860
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [6]:
!pip install tensorflow==2.8.0

In [7]:
!python models/research/object_detection/builders/model_builder_tf2_test.py

2023-05-03 16:32:17.712183: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-05-03 16:32:17.712731: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-03 16:32:29.532229: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-05-03 16:32:29.532802: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2023-05-03 16:32:29.533293: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2023-05-03 16:32:29.534207: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cu

In [1]:
import os
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import label_map_util
from object_detection.builders import model_builder

In [15]:
CONFIG_PATH = "..\Saved_Model\mymodel\pipeline_file.config"
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

CHECKPOINT_PATH = r"..\Saved_Model\training\ckpt-4"
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(CHECKPOINT_PATH)

In [16]:
@tf.function
def detect_fn(image):
    images, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [17]:
import cv2 
import numpy as np

In [18]:
category_index = label_map_util.create_category_index_from_labelmap("..\Saved_Model\labelmap.pbtxt")

In [19]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while True: 
    ret, frame = cap.read()
    if ret is True:
        image_np = np.array(frame)
    
        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)
    
        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        label_id_offset = 1
        image_np_with_detections = image_np.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detections,
                    detections['detection_boxes'],
                    detections['detection_classes']+label_id_offset,
                    detections['detection_scores'],
                    category_index,
                    use_normalized_coordinates=True,
                    max_boxes_to_draw=3,
                    min_score_thresh=.4,
                    agnostic_mode=False)
        print(detections['detection_scores'])

        cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break

[0.05279559 0.04824114 0.03365353 0.02824962 0.02775156 0.02772167
 0.02733573 0.02628115 0.02535746 0.02528328 0.02511656 0.02480027
 0.02477765 0.02401891 0.02338091 0.02337694 0.02325168 0.02278042
 0.02254418 0.02216345 0.02211514 0.02156496 0.0213654  0.02120209
 0.02100566 0.02073219 0.02025905 0.01993239 0.01991716 0.01988012
 0.01986739 0.01975101 0.0195449  0.01946989 0.01944867 0.01910064
 0.0189698  0.01853588 0.01842085 0.0182426  0.01820725 0.0180223
 0.01799589 0.01792717 0.01789704 0.01741362 0.01711777 0.01696718
 0.016754   0.01663375 0.01658413 0.01641324 0.01545054 0.01516277
 0.01500294 0.0149115  0.01422456 0.01418149 0.01374561 0.01374531
 0.01365727 0.01339111 0.0133889  0.01337188 0.01324958 0.01314035
 0.01304728 0.01303226 0.01297843 0.01297244 0.01289004 0.01282337
 0.01280871 0.01269451 0.01265517 0.01261857 0.01261532 0.01251543
 0.01247051 0.01246545 0.01242754 0.01236895 0.01228058 0.01216701
 0.0120858  0.01189074 0.01170856 0.01163238]
[0.05484369 0.044

[0.05851734 0.05254015 0.03333732 0.02668968 0.02636644 0.02617392
 0.02552459 0.02510622 0.02505562 0.02498084 0.02421331 0.02392176
 0.02374455 0.02311325 0.02247298 0.02237642 0.02225944 0.02221349
 0.02206823 0.02177283 0.02173722 0.02165666 0.02164891 0.02150568
 0.02143389 0.02123439 0.02117825 0.021155   0.02106926 0.02040735
 0.02039158 0.0200237  0.01976016 0.01925489 0.01903117 0.01891765
 0.0186961  0.01829779 0.01813933 0.01761439 0.017068   0.01582661
 0.01570335 0.01544401 0.01524517 0.01514578 0.01502028 0.01492104
 0.01487908 0.01482463 0.01465663 0.01458228 0.01435196 0.0143168
 0.01430169 0.0142132  0.01419809 0.01408595 0.01372162 0.01334015
 0.01331815 0.01319459 0.01314825 0.0131188  0.01287773 0.01286507
 0.01286477 0.01286209 0.01282975 0.0126203  0.01260409 0.0122577
 0.0121859  0.01195094 0.01172668 0.01166192 0.0116587  0.01153257
 0.01148465]
[0.05608487 0.04940179 0.0302242  0.02622575 0.02462956 0.02431354
 0.02381647 0.02345952 0.0230284  0.02301583 0.0223

[0.06676209 0.06478071 0.02960551 0.02736554 0.02626792 0.02602842
 0.02521417 0.02491772 0.02469876 0.02443218 0.02429911 0.0241994
 0.02359706 0.02348202 0.02320966 0.02308282 0.02284321 0.02238536
 0.02223116 0.02210566 0.02182677 0.02174771 0.02172223 0.02134258
 0.02132669 0.02124554 0.02114308 0.02056676 0.02040109 0.02020901
 0.0202032  0.01978767 0.01962796 0.01961774 0.01958719 0.01951498
 0.01931056 0.01921278 0.01913643 0.01907939 0.01907274 0.01894134
 0.01886788 0.01880866 0.01875523 0.0178647  0.01707053 0.0170629
 0.01695171 0.01601171 0.0158546  0.01564467 0.01516932 0.01479462
 0.01448068 0.01446271 0.01439387 0.01426345 0.01418176 0.01415592
 0.01397762 0.01396403 0.01333684 0.0132837  0.01314557 0.0131321
 0.01288897 0.01285827 0.01246685 0.01237175 0.01236847 0.01178738
 0.01174867 0.01163578 0.01149037 0.01147225]
[0.07150698 0.06044582 0.03435197 0.02828506 0.02757743 0.02580196
 0.02560818 0.02542067 0.02529112 0.02521172 0.02515894 0.02447927
 0.02435809 0.02359

[0.06823462 0.05941403 0.029984   0.02598825 0.02502054 0.02482688
 0.02459168 0.02433541 0.02374557 0.023496   0.02347839 0.02330986
 0.02321959 0.02320582 0.0230774  0.02238479 0.02200422 0.02181107
 0.02139804 0.02131134 0.02121085 0.02080762 0.02061546 0.02013251
 0.02010477 0.01985994 0.01966494 0.01963481 0.01961812 0.01943117
 0.01942354 0.01927936 0.01926601 0.01921678 0.0191451  0.01873505
 0.01865077 0.01863965 0.0182105  0.01818889 0.01815489 0.01807806
 0.01788369 0.01753169 0.0169771  0.01684588 0.01675451 0.0163469
 0.01626608 0.01570797 0.01499522 0.01469082 0.01453802 0.01421121
 0.01420993 0.01402524 0.01395571 0.01385912 0.01380357 0.01355913
 0.01353335 0.01351801 0.0133656  0.01335359 0.01331419 0.01326379
 0.01318386 0.01311931 0.01311737 0.01311523 0.01311401 0.01310939
 0.01309478 0.01305673 0.01301575 0.012999   0.01299775 0.01298621
 0.01287904 0.01280972 0.0128009  0.01272082 0.01265115 0.01264882
 0.01264015 0.01263469 0.0124324  0.01234418 0.01215127 0.01214

[0.06380475 0.05914491 0.03415367 0.02810171 0.02672267 0.0262084
 0.02596632 0.02534905 0.02485013 0.02479294 0.02456817 0.02441722
 0.02369305 0.0229468  0.02289283 0.02282557 0.02265421 0.02260742
 0.02183986 0.02182403 0.02165154 0.02130434 0.0209541  0.02094132
 0.02072603 0.02062261 0.02046266 0.02038038 0.02028799 0.02026474
 0.02008286 0.01992795 0.01983616 0.01957858 0.01944235 0.01926395
 0.01923358 0.01889896 0.01826733 0.01773176 0.01772588 0.01713777
 0.01662096 0.01627231 0.01607591 0.01602975 0.015737   0.01569414
 0.01544854 0.01540828 0.01487088 0.01478481 0.01450065 0.01440886
 0.01426452 0.01423636 0.01380986 0.01377228 0.01355514 0.01347756
 0.01342952 0.01340726 0.01337203 0.01336071 0.01332572 0.01316857
 0.01313111 0.01310465 0.01293293 0.01275036 0.01265299 0.01253605
 0.01252651 0.01234171 0.01226571 0.0122059  0.0121944  0.01212016
 0.01188228 0.01173112]
[0.06315854 0.05907404 0.02973354 0.02943218 0.02674004 0.02636307
 0.02625039 0.02598166 0.02557603 0.025

...

[0.07378533 0.06288308 0.02942422 0.02609709 0.02479491 0.02442145
 0.02407458 0.02359951 0.02333704 0.02319756 0.0229257  0.02271265
 0.02258331 0.02244487 0.02239022 0.02225083 0.02195206 0.02188194
 0.02184308 0.02141252 0.02127782 0.02126387 0.02093664 0.02059516
 0.02050352 0.02041984 0.02030247 0.02026203 0.01974112 0.01928899
 0.01912788 0.01886126 0.01867178 0.01861152 0.01853982 0.01840574
 0.01816413 0.01786003 0.0178268  0.01714191 0.01710048 0.01548731
 0.01533186 0.0145053  0.01439512 0.01424521 0.01413676 0.01404923
 0.01399019 0.01391852 0.01381454 0.01380894 0.01372173 0.01355812
 0.0134649  0.01343268 0.01338443 0.01338121 0.01330236 0.0132986
 0.01329687 0.01324677 0.01324433 0.01304811 0.01299164 0.01284319
 0.01282871 0.01279697 0.01273385 0.01270238 0.0126875  0.01266247
 0.01264605 0.01261997 0.01254511 0.01250592 0.01247749 0.01234502
 0.01226142 0.01206094 0.01203483 0.0118919  0.01167905 0.01157624]
[0.08319947 0.06281927 0.02820304 0.02433079 0.02380466 0.0231

[0.06919029 0.05969381 0.02871007 0.02550703 0.02425078 0.02398297
 0.02343538 0.02339464 0.02321056 0.02312881 0.02310324 0.02281213
 0.0226737  0.02265605 0.02261612 0.02253562 0.02247182 0.02227461
 0.021503   0.02144772 0.02080533 0.02057171 0.02046576 0.02035451
 0.02012897 0.02010775 0.02006182 0.01992998 0.01942196 0.01936173
 0.01918238 0.01898256 0.0185433  0.01795635 0.01772946 0.01698282
 0.0168488  0.0164842  0.01632178 0.01626718 0.01615947 0.01583692
 0.0156118  0.01491979 0.01475376 0.01472771 0.013787   0.01377273
 0.01352373 0.01329151 0.01327917 0.01319811 0.01306552 0.0130004
 0.01298285 0.01293433 0.01288027 0.01283625 0.01282811 0.01281989
 0.01279503 0.01277703 0.0127638  0.01274854 0.01267663 0.01266065
 0.01259646 0.01258603 0.01254058 0.01253647 0.01251128 0.01248583
 0.01246607 0.01244867 0.01241627 0.01222017 0.01207998 0.01168048
 0.01167333 0.0116691 ]
[0.06949395 0.06363326 0.02965042 0.02599958 0.02473533 0.02454019
 0.02450016 0.02373323 0.02357081 0.023

[0.07345963 0.06283519 0.03289977 0.02878958 0.0265733  0.02652147
 0.0262222  0.02491283 0.02487847 0.02471343 0.0244036  0.0242877
 0.02393708 0.02375805 0.0236491  0.02344617 0.02290472 0.02289435
 0.02287567 0.02257177 0.02245572 0.02210647 0.02205124 0.02199766
 0.02190566 0.02177411 0.02172735 0.02147648 0.02146822 0.02111784
 0.0209735  0.02090114 0.02086371 0.02073634 0.02062261 0.02037925
 0.02002543 0.01996961 0.01962143 0.01949009 0.01907578 0.01902997
 0.0185155  0.01829901 0.01822135 0.01816803 0.01796311 0.01785785
 0.01779667 0.01776114 0.01759091 0.01738554 0.01732755 0.01699588
 0.01678842 0.01658949 0.01641154 0.01631731 0.01511532 0.01477104
 0.01468536 0.01459724 0.01458108 0.0145219  0.01451057 0.01448378
 0.01437452 0.01434815 0.01430932 0.0139057  0.01383358 0.01355407
 0.01354787 0.01353404 0.01316744 0.01314715 0.0128642  0.01274729
 0.01270479 0.01267847 0.01259318 0.01257971 0.0125424  0.01247516
 0.01245454 0.01240084 0.0122478  0.0122357  0.01200226 0.01194

[0.0853546  0.05916014 0.03333411 0.02709603 0.02656445 0.02584472
 0.02567369 0.0256435  0.02509865 0.02503106 0.02461356 0.02418804
 0.02417412 0.02379954 0.0237489  0.0230287  0.02285877 0.02254605
 0.02254337 0.02246693 0.02215254 0.02201906 0.02193919 0.02171504
 0.02155307 0.02127099 0.02119741 0.02118912 0.0208649  0.0207172
 0.02068004 0.02055299 0.02026048 0.02009696 0.02004981 0.01990834
 0.01976544 0.0196723  0.01919758 0.01917043 0.01911294 0.01894346
 0.01888454 0.0188815  0.01860762 0.01860714 0.01845765 0.01820576
 0.01811916 0.01808816 0.01808289 0.0180825  0.01806638 0.01782778
 0.01782236 0.01778263 0.01777989 0.01777229 0.01759112 0.01753744
 0.01752168 0.01746741 0.01730949 0.01716724 0.01664913 0.01623398
 0.01584822 0.01573974 0.01566702 0.015486   0.01529765 0.01526019
 0.015237   0.01471555 0.01428133 0.01371139 0.01370353 0.01368251
 0.01357749 0.01354232 0.0135344  0.0132322  0.01320985 0.01320419
 0.01318944 0.01315895 0.01299644 0.01294848 0.0129157  0.01288

[0.05745432 0.04988265 0.03232548 0.028274   0.02664757 0.02561364
 0.02542827 0.02542043 0.02503055 0.02491251 0.02428508 0.02360672
 0.02344483 0.02336919 0.02320454 0.02309754 0.02263421 0.02227795
 0.02211806 0.02197695 0.02186251 0.02156988 0.02132988 0.02115548
 0.02096498 0.02094549 0.02089202 0.02086416 0.0207991  0.02071127
 0.02058429 0.01942933 0.01928231 0.01919636 0.01905429 0.01874876
 0.01836208 0.01831126 0.01788014 0.01787153 0.01785883 0.01781145
 0.01777592 0.01753682 0.01687756 0.01670608 0.01662922 0.01637423
 0.01563114 0.01513958 0.01488906 0.01488793 0.01487568 0.01481697
 0.01467866 0.01452169 0.01450399 0.01446691 0.01427758 0.01423454
 0.01394629 0.01356602 0.01347399 0.01344097 0.01333985 0.01332191
 0.01329672 0.01328418 0.01326105 0.01306215 0.01296917 0.01278841
 0.01262206 0.01259744 0.01257563 0.01254427 0.01232201 0.01223785
 0.01222324 0.01203412 0.01202032 0.01195309 0.01171896 0.01170978
 0.01166049 0.01156983]
[0.061028   0.05489245 0.0282796  0.02

In [14]:
cap.release()
cv2.destroyAllWindows()